<a href="https://colab.research.google.com/github/basu-a/ML_UChicago/blob/master/ML_Assignment_7_Gcloud1_JSS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="display:block">
    <div style="width: 15%; display: inline-block; text-align: left;">
        <img src="https://d3qi0qp55mx5f5.cloudfront.net/shared-resources/i/template/uc_wordmark_hires.gif" style="height:40px; margin-left:0px" />
    </div>
    <div style="width: 58%; display: inline-block">
        <h1  style="text-align: center">Assignment 7 - Machine Learning</h1>
        <div style="width: 100%; text-align: center; display: inline-block;"><i>Authors: Basuraj Agrawal</i> </div>
    </div>
    <div style="width: 25%; text-align: right; display: inline-block;">
        <div style="width: 100%; text-align: left; display: inline-block;">
            <time datetime="2016-11-21" pubdate>Nov 21, 2019</time>
        </div>
    </div>
    <hr> </hr>
</div>


In [0]:
import sys
import os
import json
import pandas
import numpy
import optparse

from keras.callbacks import TensorBoard
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from collections import OrderedDict


Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
datasets = "/content/gdrive/My Drive/datasets/msca/"

### 1. Data Processing

In [0]:
dataframe = pandas.read_csv(datasets+"dev-access.csv", engine='python', quotechar='|', header=None)

In [0]:
dataframe.head(1)

,0,1
0,"{""timestamp"":1502738402847,""method"":""post"",""qu...",0


In [0]:
dataset = dataframe.values
dataset.shape

(26773, 2)

In [0]:
X = dataset[:,0]
Y = dataset[:,1]

In [0]:
for index, item in enumerate(X):
    # Quick hack to space out json elements
    reqJson = json.loads(item, object_pairs_hook=OrderedDict)
    del reqJson['timestamp']
    del reqJson['headers']
    del reqJson['source']
    del reqJson['route']
    del reqJson['responsePayload']
    X[index] = json.dumps(reqJson, separators=(',', ':'))


In [0]:
tokenizer = Tokenizer(filters='\t\n', char_level=True)
tokenizer.fit_on_texts(X)

# we will need this later
num_words = len(tokenizer.word_index)+1
X = tokenizer.texts_to_sequences(X)


In [0]:
max_log_length = 1024
X_processed = sequence.pad_sequences(X, maxlen=max_log_length)

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_processed, Y, test_size=0.25)

### 2. Model 1 - RNN

In [0]:
from keras import models
from keras import layers

model_1 = models.Sequential()
model_1.add(Embedding(num_words, 32, input_length = max_log_length))
model_1.add(layers.SimpleRNN(units = 32, activation  = 'relu'))
model_1.add(layers.Dense(1, activation = 'sigmoid'))


In [0]:
model_1.compile(optimizer = 'adam',
             loss = 'binary_crossentropy',
             metrics = ['accuracy'])




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
model_1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1024, 32)          2016      
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 32)                2080      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 4,129
Trainable params: 4,129
Non-trainable params: 0
_________________________________________________________________


In [0]:
model_1.fit(X_train, y_train, epochs = 3, batch_size = 128, validation_split = 0.25)




Train on 15059 samples, validate on 5020 samples
Epoch 1/3





15059/15059 [==============================] - 27s 2ms/step - loss: 0.4838 - acc: 0.7692 - val_loss: 0.1796 - val_acc: 0.9847
Epoch 2/3
15059/15059 [==============================] - 26s 2ms/step - loss: 0.2314 - acc: 0.9260 - val_loss: 0.1171 - val_acc: 0.9777
Epoch 3/3
15059/15059 [==============================] - 26s 2ms/step - loss: 0.0863 - acc: 0.9821 - val_loss: 0.1761 - val_acc: 0.9657


In [0]:
model_1.evaluate(X_test, y_test, batch_size = 128)

6694/6694 [==============================] - 4s 527us/step


[0.15809913999577727, 0.9669853599170517]

### 3) Model 2 - LSTM + Dropout Layers:



In [0]:
model_2 = models.Sequential()
model_2.add(Embedding(num_words, 32, input_length = max_log_length))
model_2.add(layers.LSTM(units = 64, recurrent_dropout=0.5))
model_2.add(layers.Dropout(0.5))
model_2.add(layers.Dense(1, activation = 'sigmoid'))


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
model_2.compile(optimizer = 'adam',
             loss = 'binary_crossentropy',
             metrics = ['accuracy'])

In [0]:
model_2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 1024, 32)          2016      
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                24832     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 26,913
Trainable params: 26,913
Non-trainable params: 0
_________________________________________________________________


In [0]:
model_2.fit(X_train, y_train, epochs = 3, batch_size = 128, validation_split = 0.25)

Train on 15059 samples, validate on 5020 samples
Epoch 1/3
15059/15059 [==============================] - 198s 13ms/step - loss: 0.5915 - acc: 0.6749 - val_loss: 0.4541 - val_acc: 0.8847
Epoch 2/3
15059/15059 [==============================] - 200s 13ms/step - loss: 0.3432 - acc: 0.8702 - val_loss: 0.2000 - val_acc: 0.9512
Epoch 3/3
15059/15059 [==============================] - 201s 13ms/step - loss: 0.2795 - acc: 0.9108 - val_loss: 0.1878 - val_acc: 0.9466


In [0]:
model_2.evaluate(X_test, y_test, batch_size = 128)

6694/6694 [==============================] - 17s 3ms/step


[0.18620988326372587, 0.9499551836397885]

#### 4) Recurrent Neural Net Model 3: Build Your Own Network

In [0]:
model_3 = models.Sequential()
model_3.add(Embedding(num_words, 64, input_length = max_log_length))
model_3.add(layers.LSTM(units = 64, recurrent_dropout=0.5))
model_3.add(layers.Dropout(0.5))
model_3.add(layers.Dense(32, activation = 'relu'))
model_3.add(layers.Dense(1, activation = 'sigmoid'))

In [0]:
model_3.compile(optimizer = 'rmsprop',
             loss = 'binary_crossentropy',
             metrics = ['accuracy'])


In [0]:
model_3.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 1024, 64)          4032      
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 33        
Total params: 39,169
Trainable params: 39,169
Non-trainable params: 0
_________________________________________________________________


In [0]:
model_3.fit(X_train, y_train, epochs = 3, batch_size = 128, validation_split = 0.25)

Train on 15059 samples, validate on 5020 samples
Epoch 1/3
15059/15059 [==============================] - 218s 14ms/step - loss: 0.5368 - acc: 0.7268 - val_loss: 0.3195 - val_acc: 0.9392
Epoch 2/3
15059/15059 [==============================] - 222s 15ms/step - loss: 0.2596 - acc: 0.9202 - val_loss: 0.1420 - val_acc: 0.9657
Epoch 3/3
15059/15059 [==============================] - 220s 15ms/step - loss: 0.1694 - acc: 0.9600 - val_loss: 0.1248 - val_acc: 0.9705


In [0]:
model_3.evaluate(X_test, y_test, batch_size = 128)


6694/6694 [==============================] - 20s 3ms/step


[0.1009964926214513, 0.9777412607237443]

### Part 5: Conceptual Questions:¶

#### 5) Explain the difference between the relu activation function and the sigmoid activation function.¶

Sigmoid functions are used in machine learning for the logistic regression and basic neural network implementations and they are the introductory activation units. Sigmoid functions are one of the most widely used activation functions. The main reason why we use sigmoid function is because it exists between (0 to 1). Therefore, it is especially used for models where we have to predict the probability as an output. Since probability of anything exists only between the range of 0 and 1, sigmoid is a better choice. But for advanced NN, Sigmoid functions are not preferable due to multiple drawbacks, like vanishing gradients. There are ways to work around this problem and sigmoid is still very popular in classification systems.

Relu is recently started to gain popularity amongst activation functions. Most Deep Learning applications right now make use of Rele instead of Logistic Activation functions for Computer Vision, Speech Recognition and Deep Neural Networks etc. Relu has output 0 if the input is less than 0, and raw output otherwise. That is, if the input is greater than 0, the output is equal to the input. The operation of ReLU is closer to the way our biological neurons work. Relu is non-linear and has two additional major benefits compared to sigmoid: sparsity and a reduced likelihood of vanishing gradient. For larger Neural Networks, the speed of building models based off on Relu is very fast opposed to using Sigmoid




##### 6) Describe what one epoch actually is (epoch was a parameter used in the .fit() method).

The number of epochs is a hyperparameter that defines the number times that the learning algorithm will work through the entire training dataset. One epoch means that each sample in the training dataset has had an opportunity to update the internal model parameters (1 Epoch = 1 Forward pass + 1 Backward pass for ALL training samples) An epoch is comprised of one or more batches. For example, an epoch that has one batch is called the batch gradient descent learning algorithm. We can think of a for-loop over the number of epochs where each loop proceeds over the training dataset. Within this for-loop is another nested for-loop that iterates over each batch of samples, where one batch has the specified “batch size” number of samples. The number of epochs is traditionally large, often hundreds or thousands, allowing the learning algorithm to run until the error from the model has been sufficiently minimized.



7) Explain how dropout works (you can look at the keras code and/or documentation) for (a) training, and (b) test data sets.

Dropout is one of the most popular regularization techniques for deep neural networks. At every training step, every neuron (including the input neurons, but always excluding the output neurons) has a probability p of being temporarily “dropped out,” meaning it will be entirely ignored during this training step, but it may be active during the next step. The hyperparameter p is the dropout rate, and it is typically set to 50%. After training, neurons don’t get dropped anymore.

Neurons trained with dropout cannot co-adapt with their neighboring neurons; they have to be as useful as possible on their own. They also cannot rely excessively on just a few input neurons; they must pay attention to each of their input neurons. They end up being less sensitive to slight changes in the inputs. In the end you get a more robust network that generalizes better.

The power of dropout is that a unique neural network is generated at each training step. Since each neuron can be either present or absent, there is a total of 2N possible networks. This is such a huge number that it is virtually impossible for the same neural network to be sampled twice. Once you have run a 10,000 training steps, you have essentially trained 10,000 different neural networks (each with just one training instance). These neural networks are obviously not independent since they share many of their weights, but they are nevertheless all different. The resulting neural network can be seen as an averaging ensemble of all these smaller neural networks.



8) Explain why problems such as this homework assignment are better modeled with RNNs than CNNs. What type of problem will CNNs outperform RNNs on?


This homework assignment is "identifying breach by predicting suspicious access" using the Logfile, and RNN is designed to work with sequence prediction problems, therefore this assignment is better modeled with RNNs than CNNs.

(RNN also better than CNN where data contains temporal properties, such as a time series and where the data is context sensitive, as in the case of sentence completion.)

They are both different architecture’s of neural nets that perform well on different types of data but some types of data can be processed by either architecture. Examples of this are image classification and text classification, where both systems have been effective. Moreover, some deep learning applications may benefit from the combination of the two architectures.

RNNs are good with series of data (one thing happens after another) and are used a lot in problems that can be framed as “what will happen next given…” while CNNs are especially good at problems like image classification - the general field of computer vision. CNN outperforms RNN on things like medical image analysis, image recognition, face detection and recognition systems, and full motion video analysis. Besides, CNN has been the subject of research and testing for other tasks, and it has been effective in solving traditional Natural Language Processing (NLP) tasks. Specifically, it has achieved very impressive results in semantic parsing, sentence modeling, and search query retrieval. CNNs have been employed in the field of drug discovery. It discovers chemical features, and has been used to predict novel biomolecules for combating disease. Finally CNNs have also been applied to more traditional machine learning problems, such as game playing.



9) Explain what RNN problem is solved using LSTM and briefly describe how.


LSTM is basically considered to avoid the problem of vanishing gradient in RNN (short term memory) - reason why some people say RNN has a bad memory. LSTM has internal mechanisms called gates that can regulate the flow of information. In other words, LSTM networks have some internal contextual state cells that act as long-term or short-term memory cells. These gates can learn which data in a sequence is important to keep or throw away. By doing that, it can pass relevant information down the long chain of sequences to make predictions

The output of the LSTM network is modulated by the state of these cells which is a very important property when we need the prediction of the neural network to depend on the historical context of inputs, rather than only on the very last input. In other words, LSTM networks manage to keep contextual information of inputs by integrating a loop that allows information to flow from one step to the next. (LSTM predictions are always conditioned by the past experience of the network’s inputs). LSTM networks remembers and when to forget, through their forget gate weights.

### End of Document